In [3]:
import time
import numpy as np
import pandas as pd
import community as community_louvain
import networkx as nx
import pycombo
from sklearn.cluster import SpectralClustering

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)

import warnings
warnings.filterwarnings("ignore") 

In [4]:
cuda = torch.cuda.is_available()
weight_decay = 1e-1
epochs = 1000
seed = 17

In [5]:
def normalize(adj):
    adj = torch.FloatTensor(adj)
    adj_id = torch.FloatTensor(torch.eye(adj.shape[1]))
    adj = adj + adj_id
    rowsum = torch.FloatTensor(adj.sum(-1))
    degree_mat_inv_sqrt = torch.diag_embed(torch.float_power(rowsum,-0.5), dim1=-2, dim2=-1).float()
    adj_norm = torch.mm(degree_mat_inv_sqrt.T, torch.mm(adj, degree_mat_inv_sqrt))
    return adj_norm

def modularity_matrix(adj):
    w_in = adj.sum(dim=0, keepdim=True)
    w_out = adj.sum(dim=1, keepdim=True)
    T = w_out.sum()
    Q = adj / T - w_out * w_in / T ** 2
    return Q

def modularity(C,Q):
    Q1 = torch.mm(C.T, Q)
    Q2 = torch.mm(Q1, C)
    M = torch.trace(Q2)
    return -M

def doublerelu(x):
    return torch.clamp(x, 0, 1)

In [6]:
def is_binary(C):
    sums = C.sum(dim=1, keepdim=True)
    maxs = C.max(dim=1, keepdim=True).values
    mins = C.min(dim=1, keepdim=True).values
    zeros = torch.zeros(sums.shape)
    ones = torch.ones(sums.shape)
    if torch.all(sums == ones) and torch.all(maxs == ones) and torch.all(mins == zeros):
        return True, "all binary"
    else:
        return False, "{} non-binary out of {}".format(torch.sum(~((sums == ones) & (maxs == ones) & (mins == zeros))).item(), C.shape[0])

def test_is_binary():
    C = np.array([[0,0,1],[0.999,0.001,0],[1,0,0]])
    C = torch.FloatTensor(C)
    print(is_binary(C))
    C = np.array([[0,0,1],[1,0.000,0],[1,0,0]])
    C = torch.FloatTensor(C)
    print(is_binary(C))
    C = np.array([[-0.5,0.5,1],[1,0.000,0],[1,0,0]])
    C = torch.FloatTensor(C)
    print(is_binary(C))

test_is_binary()

(False, '1 non-binary out of 3')
(True, 'all binary')
(False, '1 non-binary out of 3')


In [7]:
def move_gains_old_slow_reliable_version(C, Q):
    node_max_gain = np.zeros([C.shape[0]])
    is_bin, ratio_nonbin = is_binary(C)
    if not is_bin:
        print("community is not binary")
        return ratio_nonbin, "{} nodes could be moved with total gain {}".format(sum(node_max_gain > 0), node_max_gain.sum())
    current_modularity = -modularity(C,Q)
    for node in range(C.shape[0]):
        current_community = C[node, :].max(0).indices.item()
        C[node, current_community] = 0
        for new_community in range(C.shape[2]):
            if new_community == current_community:
                continue
            C[node, new_community] = 1
            new_modularity = -modularity(C,Q)
            diff = current_modularity - new_modularity
            diff2 = Q[node,:] @ (C[0, :, current_community] - C[:, new_community]) + Q[node,node] + \
                    (C[:, current_community] - C[:, new_community]) @ Q[:,node] + Q[node,node]
            C[node, new_community] = 0
            if abs(diff.item() - diff2.item()) > 1e-6:
                print("diff != diff2", diff, diff2, node, new_community)
            if diff2 < 0:
                print("move node {} from {} to {} to gain {}".format(node, current_community, new_community, -diff2.item()))
                node_max_gain[node] = max(node_max_gain[node], -diff2.item())
        C[node, current_community] = 1
    return ratio_nonbin, "{} nodes could be moved with total gain {}".format(sum(node_max_gain > 0), node_max_gain.sum())

def move_gains(C, Q, verbosity=2):
    node_max_gain = np.zeros([C.shape[0]])
    non_bin = 0
    moveable_nonbin = 0
    #is_bin, ratio_nonbin = is_binary(C)
    #if not is_bin:
    #    print("community is not binary")
    #    return ratio_nonbin, "{} nodes could be moved with total gain {}".format(sum(node_max_gain > 0), node_max_gain.sum())
    Q2 = (Q + Q.T) * (1 - torch.eye(Q.shape[0]))
    for node in range(C.shape[0]):
        current_community = C[node, :].max(0).indices.item()
        max_prob = C[node, current_community].item()
        if max_prob < 1:
            non_bin += 1
            current_modularity = -modularity(C, Q)
            for new_community in range(C.shape[1]):
                old_val = torch.clone(C[node, :])
                C[node, :] = torch.zeros([C.shape[1]])
                C[node, new_community] = 1
                new_modularity = -modularity(C,Q)
                diff = new_modularity - current_modularity
                C[node, :] = old_val
                if verbosity > 1 and diff > 0:
                    print("move node {} from {} (curr. prob. {:.2f}) to {} (curr. prob. {:.2f}) to gain {}".format(
                        node, current_community, old_val[current_community], new_community, old_val[new_community], diff.item()))
                    node_max_gain[node] = max(node_max_gain[node], diff.item())
            if node_max_gain[node] > 0:
                moveable_nonbin += 1
        else:
            diffs = Q2[node, :] @ (C - C[:, [current_community]])
            new_community = diffs.max(0).indices.item()
            node_max_gain[node] = diffs[new_community].item()
            if verbosity > 1 and node_max_gain[node] > 0:
                print("move node {} from {} to {} to gain {}".format(node, current_community, new_community, node_max_gain[node]))
    ratio_nonbin = "{} non-binary out of {}, {} moveable".format(non_bin, C.shape[1], moveable_nonbin)
    return ratio_nonbin, "{} nodes could be moved with total gain {}".format(sum(node_max_gain > 0), node_max_gain.sum())

def test_move_gains():
    Q = [[1, 2, -1],
        [2, 5, 1],
        [-1,1, 10]]
    Q = torch.FloatTensor(Q)
    C = [[0.3, 0.3, 0.4],
        [1, 0, 0],
        [0, 0, 1]]
    C = torch.FloatTensor(C)
    print(move_gains(C, Q))
    print("try simple:")
    move_gains_old_slow_reliable_version(C, Q)

test_move_gains()

move node 0 from 2 (curr. prob. 0.40) to 0 (curr. prob. 0.30) to gain 4.2599992752075195
move node 0 from 2 (curr. prob. 0.40) to 1 (curr. prob. 0.30) to gain 0.25999927520751953
move node 1 from 0 to 2 to gain 2.4000000953674316
move node 2 from 2 to 0 to gain 2.200000047683716
('1 non-binary out of 3, 1 moveable', '3 nodes could be moved with total gain 8.859999418258667')
try simple:
community is not binary


In [8]:
def perturb(A):
    p = 0.4
    #epsilon = torch.rand(A.shape).uniform_(0, p) - torch.rand(A.shape).uniform_(0, p)
    #return torch.clip(A + epsilon,0,1)
    epsilon = p * torch.rand(A.shape)
    C = A + epsilon
    return C / C.sum(dim=-1, keepdim=True)

In [9]:
def spectralPartition(A, nb_comm):
    clustering = SpectralClustering(n_clusters=nb_comm,assign_labels='discretize',random_state=0).fit(A)
    clusters = list(clustering.labels_)
    partitions = torch.tensor(clusters)
    return torch.nn.functional.one_hot(partitions)

In [10]:
def test_spectral():
    G = nx.Graph()
    G.add_nodes_from(range(8))
    G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7),
        (2, 5)])
    print(nx.linalg.algebraicconnectivity.fiedler_vector(G))

def test_spectral2():
    Gs = [(nx.karate_club_graph(), 'karate')]
    net_names = ["David Copperfield","Jazz Musicians","Dolphins Social Network"]
    for net_name in net_names:
        Gs.append((nx.read_pajek('../data/'+net_name+'.net').to_undirected(), net_name))
    for G, net_name in Gs:
        A = torch.FloatTensor(nx.to_numpy_array(G))
        p = spectralPartition(A, 4)
        Q = modularity_matrix(A)
        #print(p.argmax(dim=1))
        print()
        print(-modularity(p.float(), Q))
        clusters = SpectralClustering(n_clusters=4,affinity='precomputed',assign_labels='discretize',random_state=0).fit_predict(A)
        partitions = torch.tensor(clusters)
        #print(clusters)
        print(-modularity(torch.nn.functional.one_hot(partitions).float(), Q))
        clusters2 = SpectralClustering(n_clusters=4,affinity='precomputed',random_state=0).fit_predict(A)
        partitions2 = torch.LongTensor(clusters)
        #print(clusters)
        print(-modularity(torch.nn.functional.one_hot(partitions).float(), Q))

test_spectral2()


tensor(0.3402)
tensor(0.4198)
tensor(0.4198)

tensor(-0.0670)
tensor(0.2012)
tensor(0.2012)

tensor(0.0508)
tensor(0.4409)
tensor(0.4409)

tensor(0.2680)
tensor(0.5005)
tensor(0.5005)


In [11]:
def get_features_orig(G, method, seed=seed):
    mod_before_perturb = -1
    adj = nx.to_numpy_array(G)
    adj = torch.FloatTensor(adj)
    Q = modularity_matrix(adj)
    #Combo
    combo_partition, combo_mod = pycombo.execute(G, random_seed=seed)
    nb_comm = max(combo_partition.values())+1
    if method == "combo":
        communities =  np.array(list(combo_partition.values())).reshape(-1)
        orig_partition = np.eye(nb_comm)[communities]
        features = torch.FloatTensor(orig_partition)
        mod_before_perturb = combo_mod
    elif method == "louvain":
        partition = community_louvain.best_partition(G)
        #get binary matrix of the partition
        nb_community = max(list(partition.values())) + 1
        communities =  np.array(list(partition.values())).reshape(-1)
        orig_partition = np.eye(nb_community)[communities]
        features = torch.FloatTensor(orig_partition)
        mod_before_perturb = -modularity(features, Q)
        features = perturb(features)
    elif method == "spectral":
        features = spectralPartition(adj, nb_comm).float()
        mod_before_perturb = -modularity(features, Q)
    else:
        np.random.seed(seed)
        communities =  np.random.random_integers(0, nb_comm - 1, len(G))
        orig_partition = np.eye(nb_comm)[communities]
        features = torch.FloatTensor(orig_partition)
        mod_before_perturb = -modularity(features, Q)
    return features, adj, Q, mod_before_perturb, combo_mod

def get_features(G):
    mod_before_perturb = -1
    adj = nx.to_numpy_array(G)
    adj = np.expand_dims(adj, axis=0)
    Q = torch.FloatTensor(adj)
    #Combo
    combo_partition, combo_mod = pycombo.execute(G, treat_as_modularity=True, random_seed=seed)
    orig_partition = np.array([[0,0,1],[0,0,1],[1,0,0],[1,0,0],[1,0,0],[1,0,0]])
    #orig_partition = np.array([[0.9,0,0.1],[0.9,0.1,0],[1,0,0]])
    features = torch.FloatTensor(orig_partition)
    mod_before_perturb = -modularity(features, Q)
    return features, adj, Q, mod_before_perturb, combo_mod

In [12]:
class GNN_simplest(nn.Module):
    def __init__(self, features):
        super(GNN_simplest, self).__init__()
        self.C = nn.Parameter(features)

    def forward(self):
        newC = self.C
        #newC = newC / newC.sum(dim=-1, keepdim=True) #normalize st sum = 1
        newC = 1.0 + newC - newC.max(dim=-1, keepdim=True).values
        newC = torch.clamp(newC, 0, 1)
        #newC = F.relu(newC)
        newC = newC / newC.sum(dim=-1, keepdim=True) #normalize st sum = 1
        return newC

class GNNLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.0):
        super(GNNLayer, self).__init__()
        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        if out_features > in_features:
            weight1_rand = torch.empty(in_features, out_features - in_features)
            torch.nn.init.xavier_uniform_(weight1_rand, gain=1.0)
            weight1_eye[:, in_features:out_features] = weight1_rand
            self.weight1 = nn.Parameter(weight1_eye)
        elif out_features < in_features:
            weight1_rand = torch.empty(in_features - out_features, out_features)
            torch.nn.init.xavier_uniform_(weight1_rand, gain=1.0)
            weight1_eye[out_features:in_features, :] = weight1_rand
            self.weight1 = nn.Parameter(weight1_eye)
        else:
            self.weight1 = nn.Parameter(weight1_eye)
        self.weight2 = nn.Parameter(torch.zeros(in_features, out_features))
        self.dropout = dropout

    def forward(self, input, adj):
        v1 = torch.mm(input, self.weight1)
        v2 = torch.mm(adj, torch.mm(input, self.weight2))
        output = v1 + v2
        output = F.dropout(output, p=self.dropout, training=self.training)
        return output

class GNNLayerDiag(nn.Module):
    def __init__(self, n_features, dropout=0.0):
        super(GNNLayerDiag, self).__init__()
        self.weight1 = nn.Parameter(0.5 * torch.ones(1, n_features))
        self.weight2 = nn.Parameter(torch.ones(1, n_features))
        self.bias = nn.Parameter(-0.5 * torch.ones(1, n_features))
        self.dropout = dropout

    def forward(self, input, adj):
        v1 = self.weight1 * input
        v2 = torch.mm(adj, input)
        v2 = self.weight2 * v2 / v2.max(dim=1, keepdim=True).values
        output = v1 + v2 + self.bias
        output = F.dropout(output, p=self.dropout, training=self.training)
        return output

class GNN_1L(nn.Module):
    def __init__(self, ndim, diag=False, dropout=0.0):
        super(GNN_1L, self).__init__()
        if diag:
            self.gc1 = GNNLayerDiag(ndim, dropout)
        else:
            self.gc1 = GNNLayer(ndim, ndim, dropout)

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        #x = nn.Softmax(dim=2)(x)
        x = 1.0 + x - x.max(dim=-1, keepdim=True).values
        x = torch.clamp(x, 0, 1)
        x = x / x.sum(dim=-1, keepdim=True) #normalize st sum = 1
        return x

class GNN_2L(nn.Module):
    def __init__(self, ndim, hidden):
        super(GNN_2L, self).__init__()
        self.gc1 = GNNLayer(ndim, hidden)
        self.gc2 = GNNLayer(hidden, ndim)

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = torch.clamp(x, 0, 1)
        x = self.gc2(x, adj)
        x = torch.clamp(x, 0, 1)
        x = x / x.sum(dim=-1, keepdim=True) #normalize st sum = 1
        return x


In [13]:
def GNNS(model_name, G, features, adj, Q, optimizer_class, n_epochs=1000, lr=0.002, seed=seed, verbosity=2):
    np.random.seed(seed)
    torch.manual_seed(seed)
    t_total = time.time()
    adj_norm = normalize(adj)
    C = features
    init_mod = torch.trace(C.T @ Q @ C)
    f0 = -0.5
    f1 = 0.5
    f2 = 1.0
    for epoch in range(n_epochs):
        t_1run = time.time()
        t = (Q @ C).max(dim=1, keepdim=True).values
        C = f0 + f1 * C + f2 * (Q @ C) / t
        C = F.relu(C)
        C = C / C.sum(dim=-1, keepdim=True) #normalize st sum = 1
        Q1 = torch.mm(C.T, Q)
        Q2 = torch.mm(Q1, C)
        loss = torch.trace(Q2)
        loss = -loss
        if epoch == 0:
            best_loss = loss
            best_C = C.data
        else:
            if loss < best_loss:
                best_loss = loss
                best_C = C.data
                best_epoch = epoch
        if verbosity > 1 and (epoch % (n_epochs//20) == 0 or epoch == epochs - 1):
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Modularity: {:.8f}'.format(-best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t_1run))
    #t = (Q @ best_C).max(dim=1, keepdim=True).values
    #best_C = f0 + f1 * best_C + f2 * (Q @ best_C) / t
    #best_loss = torch.trace(best_C.T @ Q @ best_C)
    best_loss = -best_loss
    if verbosity > 0:
        print("Optimization Finished!")
        print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    ratio_nonbin, node_max_gain = move_gains(best_C, Q, verbosity)
    return init_mod, best_loss, best_C, ratio_nonbin, node_max_gain

In [14]:
def train(model_name, G, features, adj, Q, optimizer_class, n_epochs=1000, lr=0.002, seed=seed, verbosity=2):
    np.random.seed(seed)
    torch.manual_seed(seed)
    t_total = time.time()
    adj_norm = normalize(adj)
    Q_norm = Q / abs(Q).sum(dim=1, keepdim=True)
    init_mod = torch.trace(features.T @ Q @ features)
    if model_name[:2] == '1L':
        model = GNN_1L(features.shape[1])
    elif model_name[:2] == '1D':
        model = GNN_1L(features.shape[1], diag=True)
    elif model_name[:2] == '2L':
        model = GNN_2L(features.shape[1], features.shape[1] + 3)
    elif model_name == 'simple':
        model = GNN_simplest(features)
    else:
        assert False, 'unexpected model_name'
    optimizer = optimizer_class(model.parameters(), lr=lr)
    for epoch in range(n_epochs):
        t_1run = time.time()
        optimizer.zero_grad()
        if model_name[2] == 'q':
            C = model(features, Q_norm)
        elif model_name[2] == 'a':
            C = model(features, adj_norm)
        elif model_name == 'simple':
            C = model()
        else:
            assert False, 'unexpected model_name'
        Q1 = torch.mm(C.T, Q)
        Q2 = torch.mm(Q1, C)
        loss = torch.trace(Q2)
        loss = -loss
        loss.backward()
        optimizer.step()
        if epoch == 0:
            best_loss = loss
            best_C = C.data
        else:
            if loss < best_loss:
                best_loss = loss
                best_C = C.data
                best_epoch = epoch
        if epoch % (n_epochs//20) == 0 or epoch == epochs - 1:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Modularity: {:.8f}'.format(-best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t_1run))
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    ratio_nonbin, node_max_gain = move_gains(best_C, Q)
    return init_mod,-best_loss, best_C, ratio_nonbin, node_max_gain

def run_experiment(model_name, network, optimizer_class=optim.SGD, n_epochs=1000, partition_method='spectral',
                    init_features=None, seed=seed, verbosity=2):
    print("->>starting {} optimization using {}".format(model_name, optimizer_class))
    #network = nx.Graph()
    #network.add_nodes_from(range(6))
    #network.add_edges_from([(0, 1, {'weight': 1.0}), (0, 2, {'weight': -10}), (0, 3, {'weight': 1}), (0, 4, {'weight': -10}), (0, 5, {'weight': -10}),
    #                    (1, 2, {'weight': 1.2}), (1, 3, {'weight': -10}), (1, 4, {'weight': -10}), (1, 5, {'weight': -10}),
    #                    (2, 3, {'weight': 1}), (2, 4, {'weight': -1}), (2, 5, {'weight': 0.5}),
    #                    (3, 4, {'weight': 0.5}), (3, 5, {'weight': -1})])
    features, adj, Q, mod_before_perturb, combo_mod = get_features_orig(network, partition_method, seed)
    print("Combo's modularity = {:.8f}, modularity before perturbation = {:.8f}".format(combo_mod, mod_before_perturb.item()))
    if init_features is None:
        init_features = features
    if model_name == 'GNNS':
        init_mod, finetuned_mod, features, ratio_nonbin, node_max_gain = GNNS(model_name, network, init_features, adj, Q, optimizer_class, n_epochs, verbosity=verbosity)
    else:
        init_mod, finetuned_mod, features, ratio_nonbin, node_max_gain = train(model_name, network, init_features, adj, Q, optimizer_class, n_epochs, verbosity=verbosity)
    if verbosity > 0:
        print("Starting from init. modularity = {}, achieved modularity = {}".format(init_mod, finetuned_mod))
    return features, finetuned_mod, combo_mod

In [52]:
def random_trials(model_name, num_tries=5):
    t_total = time.time()
    net_names=["karate", "David Copperfield","Jazz Musicians","Dolphins Social Network"]
    results = {'Method': '{} with {} random starts'.format(model_name, num_tries)}
    for net_name in net_names:
        print("\t\tStarting processing net {}".format(net_name))
        if net_name == 'karate':
            network = nx.karate_club_graph()
        else:
            network = nx.read_pajek('../data/'+net_name+'.net').to_undirected()

        best_mod = -1
        best_res = []
        for i in range(num_tries):
            print("\t->>Starting trial ", i)
            new_features, new_mod, combo_mod = run_experiment(model_name, network, optim.Adam, n_epochs=2000, partition_method='rand', seed=7*i)
            old_mod = -1
            while new_mod > old_mod:
                old_mod = new_mod
                new_features, new_mod, _ = run_experiment(model_name, network, optim.SGD, n_epochs=1000, init_features=new_features)
            print("starting Adam")
            old_mod = -1
            while new_mod > old_mod:
                old_mod = new_mod
                new_features, new_mod, _ = run_experiment(model_name, network, optim.Adam, n_epochs=1000, init_features=new_features)
            if best_mod < new_mod:
                best_mod = new_mod
                best_res = new_features
        print("Best mod for net {} is {}\n\n\n".format(net_name, best_mod))
        results[net_name] = best_mod.item()
    results['time'] = time.time() - t_total
    return results

## Simple

In [41]:
net_names=["karate"]#, "David Copperfield","Jazz Musicians","Dolphins Social Network"]
for net_name in net_names:
    if net_name == 'karate':
        network = nx.karate_club_graph()
    else:
        network = nx.read_pajek('../data/'+net_name+'.net').to_undirected()
    new_features, new_mod, combo_mod = run_experiment('simple', network, optim.Adam, n_epochs=2000, partition_method='spectral')

->>starting simple optimization using <class 'torch.optim.adam.Adam'>
Combo's modularity = 0.41978961, modularity before perturbation = 0.34015453
Epoch: 0001 Modularity: 0.34015453 time: 0.0005s
Epoch: 0101 Modularity: 0.35850057 time: 0.0002s
Epoch: 0201 Modularity: 0.36416149 time: 0.0002s
Epoch: 0301 Modularity: 0.36829615 time: 0.0003s
Epoch: 0401 Modularity: 0.37089938 time: 0.0002s
Epoch: 0501 Modularity: 0.37278268 time: 0.0002s
Epoch: 0601 Modularity: 0.37522066 time: 0.0002s
Epoch: 0701 Modularity: 0.38014346 time: 0.0006s
Epoch: 0801 Modularity: 0.39196938 time: 0.0002s
Epoch: 0901 Modularity: 0.40203816 time: 0.0002s
Epoch: 1000 Modularity: 0.40203816 time: 0.0002s
Epoch: 1001 Modularity: 0.40203816 time: 0.0006s
Epoch: 1101 Modularity: 0.40203816 time: 0.0002s
Epoch: 1201 Modularity: 0.40203816 time: 0.0002s
Epoch: 1301 Modularity: 0.40203816 time: 0.0002s
Epoch: 1401 Modularity: 0.40203816 time: 0.0002s
Epoch: 1501 Modularity: 0.40203816 time: 0.0002s
Epoch: 1601 Modulari

In [345]:
new_features, new_mod, combo_mod = run_experiment('simple', net_names, optim.Adam, n_epochs=2000, partition_method='spectral')
old_mod = -1
while new_mod > old_mod:
    old_mod = new_mod
    new_features, new_mod, _ = run_experiment('simple', net_names, optim.SGD, n_epochs=1000, init_features=new_features)
print("starting Adam")
old_mod = -1
while new_mod > old_mod:
    old_mod = new_mod
    new_features, new_mod, _ = run_experiment('simple', net_names, optim.Adam, n_epochs=1000, init_features=new_features)

->>starting simple optimization
Combo's modularity = 0.41978961, modularity before perturbation = 0.34015453
Epoch: 0001 Modularity: 0.34015453 time: 0.0004s
Epoch: 0101 Modularity: 0.35850057 time: 0.0002s
Epoch: 0201 Modularity: 0.36416149 time: 0.0002s
Epoch: 0301 Modularity: 0.36829615 time: 0.0002s
Epoch: 0401 Modularity: 0.37089938 time: 0.0002s
Epoch: 0501 Modularity: 0.37278268 time: 0.0002s
Epoch: 0601 Modularity: 0.37522066 time: 0.0002s
Epoch: 0701 Modularity: 0.38014346 time: 0.0002s
Epoch: 0801 Modularity: 0.39196938 time: 0.0002s
Epoch: 0901 Modularity: 0.40203816 time: 0.0002s
Epoch: 1000 Modularity: 0.40203816 time: 0.0002s
Epoch: 1001 Modularity: 0.40203816 time: 0.0009s
Epoch: 1101 Modularity: 0.40203816 time: 0.0002s
Epoch: 1201 Modularity: 0.40203816 time: 0.0002s
Epoch: 1301 Modularity: 0.40203816 time: 0.0002s
Epoch: 1401 Modularity: 0.40203816 time: 0.0002s
Epoch: 1501 Modularity: 0.40203816 time: 0.0002s
Epoch: 1601 Modularity: 0.40203816 time: 0.0002s
Epoch: 17

### Random tries

In [41]:
results_simple = random_trials('simple')

		Starting processing net karate
	->>Starting trial  0
->>starting simple optimization using <class 'torch.optim.adam.Adam'>
Combo's modularity = 0.41978961, modularity before perturbation = -0.05728139
Epoch: 0001 Modularity: -0.05728139 time: 0.0007s
Epoch: 0101 Modularity: 0.04344017 time: 0.0002s
Epoch: 0201 Modularity: 0.08304624 time: 0.0002s
Epoch: 0301 Modularity: 0.10831866 time: 0.0002s
Epoch: 0401 Modularity: 0.12574606 time: 0.0002s
Epoch: 0501 Modularity: 0.14403452 time: 0.0002s
Epoch: 0601 Modularity: 0.18510455 time: 0.0002s
Epoch: 0701 Modularity: 0.24153039 time: 0.0002s
Epoch: 0801 Modularity: 0.27550924 time: 0.0002s
Epoch: 0901 Modularity: 0.29317772 time: 0.0002s
Epoch: 1000 Modularity: 0.30781743 time: 0.0002s
Epoch: 1001 Modularity: 0.30786887 time: 0.0007s
Epoch: 1101 Modularity: 0.30892506 time: 0.0002s
Epoch: 1201 Modularity: 0.30892506 time: 0.0002s
Epoch: 1301 Modularity: 0.30892506 time: 0.0002s
Epoch: 1401 Modularity: 0.30892506 time: 0.0002s
Epoch: 1501 

In [31]:
results_simple

{'Method': 'simple with 5 random starts',
 'karate': tensor(0.4198, grad_fn=<NegBackward0>),
 'David Copperfield': tensor(0.3011, grad_fn=<NegBackward0>),
 'Jazz Musicians': tensor(0.4443, grad_fn=<NegBackward0>),
 'Dolphins Social Network': tensor(0.5268, grad_fn=<NegBackward0>)}

# 1-Layer

In [74]:
network = nx.karate_club_graph()
new_features, new_mod, combo_mod = run_experiment('1Lq', network, optim.Adam, n_epochs=2000, partition_method='spectral')

->>starting 1Lq optimization using <class 'torch.optim.adam.Adam'>
Combo's modularity = 0.41978961, modularity before perturbation = 0.34015453
Epoch: 0001 Modularity: 0.36633903 time: 0.0007s
Epoch: 0101 Modularity: 0.39324152 time: 0.0004s
Epoch: 0201 Modularity: 0.40112552 time: 0.0005s
Epoch: 0301 Modularity: 0.40203816 time: 0.0003s
Epoch: 0401 Modularity: 0.40203816 time: 0.0003s
Epoch: 0501 Modularity: 0.40203816 time: 0.0003s
Epoch: 0601 Modularity: 0.40203816 time: 0.0003s
Epoch: 0701 Modularity: 0.40203816 time: 0.0003s
Epoch: 0801 Modularity: 0.40203816 time: 0.0003s
Epoch: 0901 Modularity: 0.40203816 time: 0.0003s
Epoch: 1000 Modularity: 0.40203816 time: 0.0003s
Epoch: 1001 Modularity: 0.40203816 time: 0.0007s
Epoch: 1101 Modularity: 0.40203816 time: 0.0003s
Epoch: 1201 Modularity: 0.40203816 time: 0.0003s
Epoch: 1301 Modularity: 0.40203816 time: 0.0003s
Epoch: 1401 Modularity: 0.40203816 time: 0.0004s
Epoch: 1501 Modularity: 0.40203816 time: 0.0003s
Epoch: 1601 Modularity:

In [75]:
old_mod = -1
while new_mod > old_mod:
    old_mod = new_mod
    new_features, new_mod, _ = run_experiment('1Lq', network, optim.SGD, n_epochs=10000, init_features=new_features)

->>starting 1Lq optimization using <class 'torch.optim.sgd.SGD'>
Combo's modularity = 0.41978961, modularity before perturbation = 0.34015453
Epoch: 0001 Modularity: 0.39957136 time: 0.0006s
Epoch: 0501 Modularity: 0.40080482 time: 0.0003s
Epoch: 1000 Modularity: 0.40202838 time: 0.0003s
Epoch: 1001 Modularity: 0.40203077 time: 0.0006s
Epoch: 1501 Modularity: 0.40203816 time: 0.0003s
Epoch: 2001 Modularity: 0.40203816 time: 0.0003s
Epoch: 2501 Modularity: 0.40203816 time: 0.0003s
Epoch: 3001 Modularity: 0.40203816 time: 0.0003s
Epoch: 3501 Modularity: 0.40203816 time: 0.0003s
Epoch: 4001 Modularity: 0.40203816 time: 0.0003s
Epoch: 4501 Modularity: 0.40203816 time: 0.0003s
Epoch: 5001 Modularity: 0.40203816 time: 0.0003s
Epoch: 5501 Modularity: 0.40203816 time: 0.0003s
Epoch: 6001 Modularity: 0.40203816 time: 0.0003s
Epoch: 6501 Modularity: 0.40203816 time: 0.0003s
Epoch: 7001 Modularity: 0.40203816 time: 0.0003s
Epoch: 7501 Modularity: 0.40203816 time: 0.0003s
Epoch: 8001 Modularity: 0

### Random tries

1 Full Layer using modularity matrix

In [53]:
results_1FullLayerMod = random_trials('1Lq')

		Starting processing net karate
	->>Starting trial  0
->>starting 1Lq optimization using <class 'torch.optim.adam.Adam'>
Combo's modularity = 0.41978961, modularity before perturbation = -0.05728139
Epoch: 0001 Modularity: -0.05728139 time: 0.0021s
Epoch: 0101 Modularity: 0.01164936 time: 0.0003s
Epoch: 0201 Modularity: 0.03096612 time: 0.0003s
Epoch: 0301 Modularity: 0.04394842 time: 0.0002s
Epoch: 0401 Modularity: 0.05570036 time: 0.0003s
Epoch: 0501 Modularity: 0.06548599 time: 0.0002s
Epoch: 0601 Modularity: 0.07362718 time: 0.0002s
Epoch: 0701 Modularity: 0.08091836 time: 0.0002s
Epoch: 0801 Modularity: 0.08769514 time: 0.0002s
Epoch: 0901 Modularity: 0.09488131 time: 0.0002s
Epoch: 1000 Modularity: 0.10277864 time: 0.0002s
Epoch: 1001 Modularity: 0.10285838 time: 0.0006s
Epoch: 1101 Modularity: 0.11170071 time: 0.0002s
Epoch: 1201 Modularity: 0.12103376 time: 0.0002s
Epoch: 1301 Modularity: 0.13205639 time: 0.0002s
Epoch: 1401 Modularity: 0.14546598 time: 0.0003s
Epoch: 1501 Mod

1 Diagonal layer

In [54]:
results_1DiagLayerMod = random_trials('1Dq')

		Starting processing net karate
	->>Starting trial  0
->>starting 1Dq optimization using <class 'torch.optim.adam.Adam'>
Combo's modularity = 0.41978961, modularity before perturbation = -0.05728139
Epoch: 0001 Modularity: -0.01924488 time: 0.0006s
Epoch: 0101 Modularity: 0.06373797 time: 0.0003s
Epoch: 0201 Modularity: 0.09963375 time: 0.0003s
Epoch: 0301 Modularity: 0.12489013 time: 0.0003s
Epoch: 0401 Modularity: 0.15458773 time: 0.0003s
Epoch: 0501 Modularity: 0.16605614 time: 0.0003s
Epoch: 0601 Modularity: 0.17236181 time: 0.0003s
Epoch: 0701 Modularity: 0.17692281 time: 0.0003s
Epoch: 0801 Modularity: 0.18301156 time: 0.0003s
Epoch: 0901 Modularity: 0.19421744 time: 0.0003s
Epoch: 1000 Modularity: 0.21141061 time: 0.0003s
Epoch: 1001 Modularity: 0.21170196 time: 0.0006s
Epoch: 1101 Modularity: 0.22515026 time: 0.0003s
Epoch: 1201 Modularity: 0.22795093 time: 0.0003s
Epoch: 1301 Modularity: 0.23083955 time: 0.0003s
Epoch: 1401 Modularity: 0.23178679 time: 0.0003s
Epoch: 1501 Mod

# GNNS

In [16]:
network = nx.karate_club_graph()
new_features, new_mod, combo_mod = run_experiment('GNNS', network, n_epochs=200, partition_method='spectral')

->>starting GNNS optimization using <class 'torch.optim.sgd.SGD'>
Combo's modularity = 0.41978961, modularity before perturbation = 0.34015453
Epoch: 0001 Modularity: 0.35644084 time: 0.0008s
Epoch: 0011 Modularity: 0.40013891 time: 0.0001s
Epoch: 0021 Modularity: 0.40013891 time: 0.0001s
Epoch: 0031 Modularity: 0.40013891 time: 0.0001s
Epoch: 0041 Modularity: 0.40013891 time: 0.0001s
Epoch: 0051 Modularity: 0.40013891 time: 0.0001s
Epoch: 0061 Modularity: 0.40013891 time: 0.0001s
Epoch: 0071 Modularity: 0.40013891 time: 0.0001s
Epoch: 0081 Modularity: 0.40013891 time: 0.0001s
Epoch: 0091 Modularity: 0.40013891 time: 0.0001s
Epoch: 0101 Modularity: 0.40013891 time: 0.0001s
Epoch: 0111 Modularity: 0.40013891 time: 0.0001s
Epoch: 0121 Modularity: 0.40013891 time: 0.0001s
Epoch: 0131 Modularity: 0.40013891 time: 0.0001s
Epoch: 0141 Modularity: 0.40013891 time: 0.0001s
Epoch: 0151 Modularity: 0.40013891 time: 0.0001s
Epoch: 0161 Modularity: 0.40013891 time: 0.0001s
Epoch: 0171 Modularity: 

In [55]:
def random_trials_GNNS(num_tries=5):
    t_total = time.time()
    net_names=["karate", "David Copperfield","Jazz Musicians","Dolphins Social Network"]
    results = {'Method': '{} with {} random starts'.format('GNNS', num_tries)}
    results_combo = {'Method': 'Combo'}
    for net_name in net_names:
        print("\t\tStarting processing net {}".format(net_name))
        if net_name == 'karate':
            network = nx.karate_club_graph()
        else:
            network = nx.read_pajek('../data/'+net_name+'.net').to_undirected()
        best_mod = -1
        best_res = []
        for i in range(num_tries):
            print("\t->>Starting trial ", i)
            new_features, new_mod, combo_mod = run_experiment('GNNS', network, n_epochs=200, partition_method='rand', seed=7*i)
            old_mod = -1
            while new_mod > old_mod:
                old_mod = new_mod
                new_features, new_mod, _ = run_experiment('GNNS', network, n_epochs=100, init_features=new_features)
            if best_mod < new_mod:
                best_mod = new_mod
                best_res = new_features
        print("Best mod for net {} is {}\n\n\n".format(net_name, best_mod))
        results[net_name] = best_mod.item()
        results_combo[net_name] = combo_mod
    results['time'] = time.time() - t_total
    results_combo['time'] = -1
    return results, results_combo

results_GNNS, results_combo = random_trials_GNNS()

		Starting processing net karate
	->>Starting trial  0
->>starting GNNS optimization using <class 'torch.optim.sgd.SGD'>
Combo's modularity = 0.41978961, modularity before perturbation = -0.05728139
Epoch: 0001 Modularity: -0.05417300 time: 0.0001s
Epoch: 0011 Modularity: 0.12610690 time: 0.0001s
Epoch: 0021 Modularity: 0.21892895 time: 0.0001s
Epoch: 0031 Modularity: 0.22017200 time: 0.0001s
Epoch: 0041 Modularity: 0.22017303 time: 0.0001s
Epoch: 0051 Modularity: 0.22017303 time: 0.0001s
Epoch: 0061 Modularity: 0.22017303 time: 0.0001s
Epoch: 0071 Modularity: 0.22017303 time: 0.0001s
Epoch: 0081 Modularity: 0.22017303 time: 0.0001s
Epoch: 0091 Modularity: 0.22017303 time: 0.0001s
Epoch: 0101 Modularity: 0.22017303 time: 0.0001s
Epoch: 0111 Modularity: 0.22017303 time: 0.0001s
Epoch: 0121 Modularity: 0.22017303 time: 0.0001s
Epoch: 0131 Modularity: 0.22017303 time: 0.0001s
Epoch: 0141 Modularity: 0.22017303 time: 0.0001s
Epoch: 0151 Modularity: 0.22017303 time: 0.0001s
Epoch: 0161 Modu

# Results


In [57]:
results = pd.DataFrame(columns=['Method']+net_names+['time'])
results = results.append(results_combo, ignore_index=True)
results = results.append(results_GNNS, ignore_index=True)
results = results.append(results_simple, ignore_index=True)
results = results.append(results_1DiagLayerMod, ignore_index=True)
results = results.append(results_1FullLayerMod, ignore_index=True)
results

,Method,karate,David Copperfield,Jazz Musicians,Dolphins Social Network,time
0,Combo,0.419790,0.308742,0.444469,0.526799,-1
1,GNNS with 5 random starts,0.416968,0.296378,0.444132,0.521361,10.108678
2,simple with 5 random starts,0.419790,0.301149,0.444259,0.526799,184.769144
3,1Dq with 5 random starts,0.402038,0.290199,0.444315,0.510660,303.862727
4,1Lq with 5 random starts,0.396183,0.262555,0.432380,0.482002,277.874147


## Best results that I've seen so far
### rand GNNS
- Best mod for net karate is 0.41696786880493164
- Best mod for net David Copperfield is 0.29637816548347473
- Best mod for net Jazz Musicians is 0.4441315531730652
- Best mod for net Dolphins Social Network is 0.5213608741760254

### rand simple
- Best mod for net karate is 0.4197896718978882
- Best mod for net David Copperfield is 0.30114877223968506
- Best mod for net Jazz Musicians is 0.44425898790359497
- Best mod for net Dolphins Social Network is 0.5267987251281738


### rand 1L adj_norm
- Best mod for net karate is 0.3878205418586731
- Best mod for net David Copperfield is 0.23354023694992065
- Best mod for net Jazz Musicians is 0.4232749938964844
- Best mod for net Dolphins Social Network is 0.4712923765182495

### rand 1L Q_norm
- Best mod for net karate is 0.39618340134620667
- Best mod for net David Copperfield is 0.26255524158477783
- Best mod for net Jazz Musicians is 0.432380348443985
- Best mod for net Dolphins Social Network is 0.48200228810310364

### rand 1L Q_norm Diag
- Best mod for net karate is 0.4020381569862366
- Best mod for net David Copperfield is 0.2917425334453583
- Best mod for net Jazz Musicians is 0.4443153738975525
- Best mod for net Dolphins Social Network is 0.5106601715087891
